In [ ]:
pip install -q yahoo_fin

In [4]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.6#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [6]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-4a', mod, True, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
28,GTLB,RROnly,28.815948,0.955752,435.84,444.20,880.04,57.07,53.130001,-0.07,8.925411
53,PLUG,RROnly,3.430836,0.936170,435.24,341.29,776.53,2.89,3.270000,0.13,8.870704
80,ZS,RROnly,71.083160,0.952128,1979.04,1707.62,3686.66,181.41,195.009995,0.07,8.837386
39,MDB,RROnly,146.861200,0.941489,3889.56,3996.96,7886.51,352.06,318.790009,-0.09,8.764775
43,MXL,RROnly,15.443330,0.914894,352.78,278.26,631.04,22.05,19.250000,-0.13,8.494272
40,MNDY,RROnly,81.922446,0.900763,1010.99,1514.85,2525.84,193.11,230.580002,0.19,8.485335
16,DOCN,RROnly,21.766258,0.900709,541.96,418.92,960.88,35.11,41.689999,0.19,8.484509
4,APP,RROnly,11.209495,0.942029,438.31,346.77,785.09,75.57,79.860001,0.06,8.454106
70,TTD,RROnly,27.373336,0.941489,698.03,616.74,1314.77,86.37,81.339996,-0.06,8.439716
46,NTNX,RROnly,14.595626,0.898936,240.99,214.02,455.01,63.22,51.570000,-0.18,8.427896


In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-3a', mod, False)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
21,ONLN,RROnly,26.954521,0.946809,273.84,363.93,637.77,37.04,40.189999,0.08,8.803191
2,ARKW,RROnly,38.069865,0.914894,603.06,783.28,1386.34,72.82,86.400002,0.19,8.701008
10,IPO,RROnly,24.502496,0.925532,288.27,324.51,612.78,36.32,40.950001,0.13,8.682488
0,ARKF,RROnly,14.202301,0.893617,225.33,188.64,413.97,26.68,30.860001,0.16,8.271277
24,SMOG,RROnly,67.602684,0.936170,871.20,622.72,1493.91,91.14,95.290001,0.05,8.085106
3,DAPP,RROnly,2.957197,0.935252,75.70,152.65,228.36,8.99,8.570000,-0.05,8.057554
7,FPX,RROnly,71.300464,0.898936,448.32,435.92,884.24,95.70,104.980003,0.10,7.978723
32,XITK,RROnly,93.324525,0.867021,718.55,780.22,1498.77,132.61,151.910004,0.15,7.783688
27,VNQ,RROnly,60.013765,0.936170,373.47,219.21,592.68,79.00,81.870003,0.04,7.765957
1,ARKK,RROnly,30.720241,0.936170,502.03,592.06,1094.08,42.00,43.740002,0.04,7.765957


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-4b', mod, True)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
14,BLDE,RROnly,2.321653,0.936170,70.63,98.68,169.31,3.39,4.930000,0.45,5.365040
25,EVGO,RROnly,2.125391,0.924528,99.48,118.95,218.42,1.73,2.310000,0.34,5.216426
49,ONDS,RROnly,0.553192,0.910828,36.11,76.09,112.19,0.84,1.140000,0.36,5.110112
33,HIVE,RROnly,1.557902,0.909574,165.10,136.24,301.34,2.71,3.640000,0.34,5.084481
55,PTON,RROnly,5.421874,0.888298,518.26,777.90,1296.15,3.03,5.230000,0.73,5.072265
6,APPS,RROnly,3.096824,0.888298,421.32,479.13,900.45,1.85,3.110000,0.68,5.061014
3,AMSC,RROnly,3.751485,0.909574,137.28,142.90,280.18,11.66,15.230000,0.31,5.058743
71,TRUP,RROnly,22.316050,0.920213,588.61,837.03,1425.64,22.45,27.090000,0.21,5.033077
48,OMQS,RROnly,0.536521,0.877660,58.71,80.54,139.25,0.44,0.810000,0.85,5.018773
63,SAVE,RROnly,4.551876,0.946809,184.08,168.29,352.38,4.04,4.410000,0.09,4.978445


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-3b', mod4, False)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
19,FDX,RROnly,104.927607,0.941489,1420.87,1193.10,2613.97,265.84,236.380005,-0.11,6.193009
21,INTC,RROnly,24.855015,0.914894,194.41,254.05,448.46,31.88,36.310001,0.14,5.927052
17,DIS,RROnly,80.991112,0.920213,625.17,723.06,1348.23,112.73,126.720001,0.12,5.908055
12,COST,RROnly,284.609627,0.882979,2757.64,986.48,3744.12,729.18,652.119995,-0.11,5.243161
3,AMD,RROnly,53.103318,0.893617,1010.24,796.38,1806.62,157.40,145.419998,-0.08,5.053191
29,NVDA,RROnly,87.438773,0.867021,3791.98,1483.75,5275.73,877.35,781.539978,-0.11,4.984112
5,AVGO,RROnly,249.159259,0.840426,4090.20,1563.70,5653.90,1344.07,1137.449951,-0.15,4.939210
31,ORCL,RROnly,49.773567,0.856383,502.96,331.78,834.74,117.21,103.550003,-0.12,4.920213
4,AMZN,RROnly,84.495117,0.888298,762.72,852.74,1615.46,179.62,167.929993,-0.07,4.749240
7,BKNG,RROnly,1420.955872,0.898936,16653.09,9136.90,25789.99,3521.08,3323.790039,-0.06,4.736575


In [ ]:
findata.EPOCHS=200
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-4b', mod5, True)[0]


In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
26,GLOB,RROnly,112.871148,0.930851,1693.07,1571.48,3264.55,180.83,229.429993,0.27,6.136722
2,ADYEY,RROnly,7.049236,0.925532,168.92,136.75,305.67,12.57,16.410000,0.31,6.130503
52,SFTBY,RROnly,17.429729,0.920213,190.50,184.68,375.18,24.98,29.309999,0.17,5.634275
34,MGA,RROnly,33.299563,0.914894,386.42,372.52,758.94,49.12,57.660000,0.17,5.533703
5,BESIY,RROnly,34.846932,0.893617,799.21,515.60,1314.81,142.54,157.330002,0.10,4.255319
48,PROSF,RROnly,28.842684,0.893617,421.15,604.06,1025.21,30.90,34.080002,0.10,4.255319
45,OTGLY,RROnly,4.456901,0.835106,69.52,89.69,159.21,7.16,8.450000,0.18,4.132894
67,YARIY,RROnly,10.871100,0.898936,75.06,106.03,181.09,14.46,15.470000,0.07,3.533435
1,ADDDF,RROnly,97.315716,0.867021,1190.71,1566.46,2757.17,245.10,269.899994,0.10,3.533435
3,BAK,RROnly,6.471445,0.914894,93.47,114.02,207.50,8.94,9.420000,0.05,3.130699


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-3b', mod6, False)[0]


In [9]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
14,EWZ,RROnly,17.131747,0.946809,149.16,141.91,291.07,32.31,30.430000,-0.06,8.581560
4,ECH,RROnly,19.541136,0.936170,132.94,102.36,235.30,27.53,26.730000,-0.03,7.234043
11,EWP,RROnly,17.681738,0.882979,105.31,68.88,174.18,31.77,29.570000,-0.07,7.158055
7,EWI,RROnly,17.991586,0.877660,124.09,84.32,208.41,36.96,34.290001,-0.07,7.028875
16,ILF,RROnly,13.984707,0.877660,107.96,76.93,184.89,28.26,26.389999,-0.07,7.028875
9,EWC,RROnly,22.039865,0.898936,119.48,73.90,193.38,37.57,35.880001,-0.05,6.968085
6,EWA,RROnly,14.332315,0.882979,76.41,56.11,132.52,24.16,22.660000,-0.06,6.879433
20,TUR,RROnly,16.600182,0.845745,118.36,141.26,259.62,41.50,37.700001,-0.09,6.743499
18,KEMQ,RROnly,11.909613,0.835106,77.83,102.32,180.15,16.16,14.610000,-0.10,6.702128
2,FLN,RROnly,10.755155,0.851064,66.98,48.25,115.23,20.13,18.639999,-0.07,6.382979


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-5a', mod7, True)[0]

In [11]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
90,TWI,RROnly,1.445475,0.914894,107.97,105.53,213.50,10.02,11.510000,0.15,8.581560
13,BXC,RROnly,6.688358,0.941489,747.98,664.97,1412.95,102.54,109.070000,0.06,8.439716
52,KHC,RROnly,24.508720,0.962766,171.98,107.60,279.58,36.35,37.610001,0.03,8.386525
87,TGT,RROnly,105.970556,0.898936,1065.22,902.00,1967.22,158.04,173.600006,0.10,7.978723
45,HSII,RROnly,18.306623,0.930851,212.70,150.98,363.68,29.83,31.070000,0.04,7.579787
66,PETS,RROnly,4.533957,0.856383,92.72,158.84,251.55,4.05,4.630000,0.14,7.537994
83,SWK,RROnly,69.806082,0.898936,504.96,682.38,1187.34,85.80,90.449997,0.05,6.968085
64,PCAR,RROnly,39.578928,0.851064,388.23,121.61,509.84,104.79,114.239998,0.09,6.855792
23,CG,RROnly,21.318837,0.909574,256.54,180.27,436.81,41.06,42.849998,0.04,6.835106
86,TEX,RROnly,13.569348,0.941489,370.68,284.07,654.75,58.35,59.639999,0.02,6.489362


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-4a', mod8, True)[0]

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
0,GLD,RROnly,152.676240,0.904255,455.75,414.71,870.46,215.14,200.750000,-0.07,7.674772
3,SLV,RROnly,14.155986,0.882979,95.00,111.35,206.35,25.03,23.360001,-0.07,7.158055
4,SOYB,RROnly,13.783014,0.803191,82.07,42.12,124.19,25.16,26.400000,0.05,4.095745
6,TLT,RROnly,82.865961,0.872340,206.72,356.01,562.73,91.39,93.529999,0.02,2.340426
7,UUP,RROnly,22.578585,0.867021,34.47,24.91,59.38,28.34,27.879999,-0.02,2.021277
1,ISHG,RROnly,63.595355,0.856383,70.18,107.16,177.34,70.00,69.160004,-0.01,0.000000
2,PDBC,RROnly,6.390457,0.760638,38.46,20.06,58.53,14.37,13.980000,-0.03,0.000000
5,TIP,RROnly,101.819081,0.840426,88.44,115.74,204.18,106.63,106.419998,-0.00,0.000000
8,VTIP,RROnly,42.259944,0.808511,29.91,14.86,44.77,47.88,47.590000,-0.01,0.000000
9,WEAT,RROnly,5.017727,0.856383,26.22,38.72,64.94,5.46,5.520000,0.01,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-3a', mod9, False)[0]

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
10,MATIC-USD,RROnly,0.406300,0.909574,15.12,16.19,31.31,0.72,0.940000,0.31,6.205265
1,AVAX-USD,RROnly,11.370416,0.861702,627.44,643.46,1270.90,36.07,41.529999,0.15,5.233029
9,LTC-USD,RROnly,47.838165,0.829787,533.97,1455.86,1989.83,82.05,99.959999,0.22,5.153357
8,LINK-USD,RROnly,5.951246,0.813830,112.98,166.19,279.17,14.30,18.209999,0.27,5.123551
14,XRP-USD,RROnly,0.338745,0.851064,5.37,5.27,10.64,0.53,0.610000,0.15,5.086120
5,ETH-USD,RROnly,1080.183199,0.824468,18812.61,21188.86,40001.47,3227.12,3867.300049,0.20,5.011398
0,ADA-USD,RROnly,0.283864,0.765957,3.26,8.30,11.56,0.48,0.720000,0.50,5.003040
4,DOT-USD,RROnly,4.340744,0.792553,122.72,226.94,349.66,6.88,8.940000,0.30,4.969605
2,BTC-USD,RROnly,17441.720140,0.829787,288350.15,227384.22,515734.36,67123.77,57063.281250,-0.15,4.792300
7,ICP-USD,RROnly,4.608763,0.702128,88.19,195.76,283.95,12.97,21.420000,0.65,4.556932


In [12]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
val-5a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,221,770 bytes  received 35 bytes  6,443,610.00 bytes/sec
total size is 195,958,352  speedup is 60.82
/content/drive/MyDrive/colab/results/com-2a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/com-2a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/com-2a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/com-2a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
